In [ ]:
# Robot Init
from naoqi import ALProxy
# Please connect to the robot and fill in the missing data
motion = ALProxy("ALMotion", "IP", PORT)
posture = ALProxy("ALRobotPosture", "IP", PORT)
leds = ALProxy("ALLeds","IP", PORT)
memory = ALProxy("ALMemory","IP", PORT)

In [ ]:
# Set robot's stiffness
motion.setStiffnesses("Body", 1.0)

In [ ]:
# Rest
motion.rest()

In [ ]:
# Set robot's posture to standing
posture.goToPosture("StandInit", 1.0)

In [ ]:
# Balancing functions (NAO ONLY)

# Activates Whole Body Balancer
isEnabled  = True
motion.wbEnable(isEnabled)

# Legs are constrained in a plane (feet stay flat on the ground)
stateName  = "Plane"
supportLeg = "Legs"
motion.wbFootState(stateName, supportLeg)

In [ ]:
# Select type of movement
# Available types: Normal, Reflect, Reflect Negative, One Joint, Smaller, Bigger, Faster, Slower

moveType = "Normal"

In [ ]:
# Initialize input
# Feel free to add artificial input for the robot
names = []
angleList = []
timeList = []
# isAbsolute is always set to true, so movement is progressive.
isAbsolute = True

In [ ]:
# Function that redirects to which mode will be used
def move(moveType, names, angleList, timeList, isAbsolute):
    if moveType == "Normal":
        moveNormal(names, angleList, timeList, isAbsolute)
    if moveType == "Reflect":
        moveReflect(names, angleList, timeList, isAbsolute)
    if moveType == "One Joint":
        moveOneJoint(names, angleList, timeList, isAbsolute)
    if moveType == "Reflect Negative":
        moveNegativeReflect(names, angleList, timeList, isAbsolute)
    if moveType == "Smaller":
        moveSmaller(names, angleList, timeList, isAbsolute)
    if moveType == "Bigger":
        moveBigger(names, angleList, timeList, isAbsolute)
    if moveType == "Slower":
        moveSlower(names, angleList, timeList, isAbsolute)
    if moveType == "Faster":
        moveFaster(names, angleList, timeList, isAbsolute)

# HEADS UP: These functions are written to use artificial input optimally, hence, a list of names, angles, and time. Due to this, the
# function used is angleInterpolation instead of setAngle. To use setAngle, just modify the input to take one name and angle at a time
# while removing the time function. This is optimal for real-time input. Both functions are throughly documented in the NAOqi site.

In [ ]:
# This mode makes it so all movement is normal.
def moveNormal(names, angleList, timeList, isAbsolute):
    if moveType == "Normal":
        motion.angleInterpolation(names, angleList, timeList, isAbsolute)

In [ ]:
# This mode gets the name of the opposing joints
def getOpposite(names):
    new_names = []
    for name in names:
        if name[0] == 'L':
            new_names.append('R'+name[1:])
        elif name[0] == 'R':
            new_names.append('L'+name[1:])
        else:
            new_names.append(name)
    return new_names

In [ ]:
# This mode makes it so all movement is reflected.
def moveReflect(names, angleList, timeList, isAbsolute):
    if moveType == "Reflect":
        names = getOpposite(names)
        motion.angleInterpolation(names, angleList, timeList, isAbsolute)

In [ ]:
# This gets the negative of opposite angles
def getOppositeNegative(angleList):
    new_angles = []
    for angle in angleList:
        new_angles.append(-angle)
    return new_angles

In [ ]:
# This mode makes it so all movement is reflected both horizontally and vertically.
def moveNegativeReflect(names, angleList, timeList, isAbsolute):
    if moveType == "Reflect Negative":
        names = getOpposite(names)
        angleList = getOppositeNegative(angleList)
        motion.angleInterpolation(names, angleList, timeList, isAbsolute)

In [ ]:
# This mode makes it so all movement is done in one joint. By default, the Head Yaw joint is selected. 
def moveOneJoint(names, angleList, timeList, isAbsolute):
    if moveType == "One Joint":
        # Select desired joint on the human
        human_joint = "HeadYaw"
        # Select desired joint on the robot
        robot_joint = "RShoulderPitch"
        indexes = []
        newAngleList = []
        start = -1
        for name in names:
            if name == human_joint:
                index = names.index(name)
                indexes.append(index)
                start+=1
        for index in indexes:
            for angle in angleList:
                a = angleList[index]
                newAngleList.append(a)
        motion.angleInterpolation(robot_joint, newAngleList, timeList, isAbsolute)

In [ ]:
# This mode makes it so all movement is constricted
def moveSmaller(names, angleList, timeList, isAbsolute):
    if moveType == "Smaller":
        newAngleList = []
        for angle in angleList:
            size = len(angle)
            for a in angle:
                newAngle = a*0.35
                newAngleList.append(newAngle)
        newAngleList = [newAngleList[i:i+size] for i in range(0, len(newAngleList), size)]
        motion.angleInterpolation(names, newAngleList, timeList, isAbsolute)

In [ ]:
# This mode makes it so all movement is expanded.
def moveBigger(names, angleList, timeList, isAbsolute):
    if moveType == "Bigger":
        newAngleList = []
        for angle in angleList:
            size = len(angle)
            for a in angle:
                newAngle = a*1.1
                newAngleList.append(newAngle)
        newAngleList = [newAngleList[i:i+size] for i in range(0, len(newAngleList), size)]
        motion.angleInterpolation(names, newAngleList, timeList, isAbsolute)

In [ ]:
# This mode makes it so all movement is slower.
def moveSlower(names, angleList, timeList, isAbsolute):
    if moveType == "Slower":
        newTimeList = []
        for time in timeList:
            size = len(time)
            for t in time:
                newTime = t*2
                newTimeList.append(newTime)
        newTimeList = [newTimeList[i:i+size] for i in range(0, len(newTimeList), size)]
        motion.angleInterpolation(names, angleList, newTimeList, isAbsolute)

In [ ]:
# This mode makes it so all movement is faster.
def moveFaster(names, angleList, timeList, isAbsolute):
    if moveType == "Faster":
        newTimeList = []
        for time in timeList:
            size = len(time)
            for t in time:
                newTime = t*0.7
                newTimeList.append(newTime)
        newTimeList = [newTimeList[i:i+size] for i in range(0, len(newTimeList), size)]
        motion.angleInterpolation(names, angleList, newTimeList, isAbsolute)

In [ ]:
move(moveType, names, angleList, timeList, isAbsolute)